# Introduction


In this tutorial ,i am going to talk about categorical variables and using original features of the data set to construct new features and why we could do that .

# Categorical features


A categorical feature takes only a limited number of values , it doesn't have continous values .A categorical variable can take on one of a limited, and usually fixed, number of possible values, assigning each individual or other unit of observation to a particular group or category. 
Consider a survey that asks how often you eat breakfast and provides four options: "Never", "Rarely", "Most days", or "Every day". In this case, the data is categorical, because responses fall into a fixed set of categories.
If people responded to a survey about which brand of car they owned, the responses would fall into categories like "Honda", "Toyota", and "Ford". In this case, the data is also categorical.
You will get an error if you try to plug these variables into most machine learning models in Python without preprocessing them first.
There are many approaches that you can use to prepare your categorical data , and i will talk about 3 of them.

But first i will import the data set to show examples about categorical features

In [9]:
#import panda library to be able to load the dataset
import pandas as pd
#read the data set and show the first 6 rows
data=pd.read_csv('https://raw.githubusercontent.com/Omarsawan/Feature-construction-and-Categorical-features-tutorial/master/ks-projects-201801.csv', encoding='latin-1')
data.head(6)

HTTPError: ignored

So in this dataset we can see that

# Approaches to handle categorical features